In [8]:
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import pandas as pd
import time
from PIL import Image
from pytesseract import pytesseract
from io import BytesIO
import base64
import pytesseract
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'  # your path may be different

In [4]:
global data

In [5]:
#get text from image
def getFeatureTextFromImg(imgUrl):
    url = imgUrl
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    r = requests.get(url, headers=headers)
    img = Image.open(BytesIO(r.content))
    text = pytesseract.image_to_string(img)
    return text

In [6]:
def getDetailsData(detailsUrl):
    #print(detailsUrl)
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0'}
    detialRequest = requests.get(detailsUrl,headers=headers)
    detailContent = detialRequest.content
    detailSoup = BeautifulSoup(detailContent,features="html.parser")
    details_container=detailSoup.find('aside',{'class':'left-side'})
    #location=details_container.find('div',{'class','location'})
    return details_container


In [7]:
global df
df=pd.DataFrame()

In [27]:
def getCarsData(page_from,page_to):
    global df
    data=[]
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0'}
    for pageNo in range(page_from,page_to):
        try:
            print(pageNo,df.shape[0])
            url='https://ksa.carswitch.com/en/saudi/used-cars/search?page={}'.format(pageNo)
            r = requests.get(url,headers=headers)
            content = r.content
            soup = BeautifulSoup(content,features="html.parser")

            #soup = BeautifulSoup(data,'html.parser')
            #print(soup)
            #get div with id main-listing-div
            soup = soup.find('section',{'id':'main-listing-div'})
            #print(soup)
            features={}
            items=soup.findAll('div',{'class':'show-price-badge'})
            #print(items)
            for index,item in enumerate(items):
                try:
                    features={}
                    title=item.find('div',{'class':'title'}).find('a').text
                    price=item.find('div',{'class':'title'}).find('span').text
                    model_year=item.find('div',{'class':'info'}).find('div',{'class','car-mileage'}).text.split(' · ')[0]
                    odo=item.find('div',{'class':'info'}).find('div',{'class','car-mileage'}).text.split(' · ')[1]
                    #get more data from details page
                    detailsUrl='https://ksa.carswitch.com'+item.find('figure',{'class','image-holder'}).find('a',href=True)['href']
                    print(detailsUrl)
                    details_html=getDetailsData(detailsUrl)
                    location=details_html.find('div',{'class','location'}).find('span').text
                    aminities_html=details_html.findAll('div',{'class','car-overview__features'})
                    car_features_html=details_html.find('div',{'class','car-overview__features-detail'}).findAll('div',{'class','feature-list__item'})
                    #print(features_html)
                    #print(features_html.length)
                    features={}
                    aminities=[]
                    #print(features_html[:-1])
                    for category in aminities_html[:-1]:
                        for index,feature in enumerate(category.findAll('div',class_='item')):
                            if len(feature["class"]) != 1:#if the div has more than one class
                                continue
                            else:
                                img=feature.find('img')
                                if img.has_attr('data-src'):
                                    imgUrl="https://ksa.carswitch.com"+img['data-src']
                                else:
                                    imgUrl="https://ksa.carswitch.com"+img['src']
                                imgText=''#getFeatureTextFromImg(imgUrl)
                                if imgText == '' and feature.has_attr('data-searchable'):
                                    aminities.append(feature['data-searchable'] )
                                else:
                                    aminities.append(imgText.replace("\n"," ").strip().replace("\x0c", "") )
                                #print(imgText)


                    features['title']=title
                    features['price']=price
                    features['model_year']=model_year
                    features['odo']=odo
                    features['aminities']=aminities
                     # car features details
                    for feature in car_features_html:
                        #print(car_features_html)
                        #print(feature)
                        name=feature.find('div',{'class':'feature-name'}).find(text=True).strip().replace(' ','_').lower()
                        value=feature.find('div',{'class':'feature-value'}).find(text=True).strip()
                        #print(name,value)
                        features[name]=value
                    #print(features)
                    df=df.append(features,ignore_index=True)
                    #print(index,title,'   |',price,'  |',model_year,'  |',odo,' |',location," |",aminities)
                    #break
                except Exception as ex :
                    print(ex)
                    continue
        except Exception as ex :
            print(ex)
            continue
            
                    

In [28]:
t0 = time.time()
getCarsData(140,142)
t1 = time.time()
print('\n\nTime comsumed: ', t1-t0)
#df.to_csv('../Data/car_switch_data_2.csv',index=False)

140 3332
https://ksa.carswitch.com/en/saudi/used-car/hyundai/tucson/2019/162820-166536
https://ksa.carswitch.com/en/saudi/used-car/hyundai/tucson/2018/162335-166051
https://ksa.carswitch.com/en/saudi/used-car/hyundai/tucson/2019/158843-162552
https://ksa.carswitch.com/en/saudi/used-car/hyundai/tucson/2017/135448-139111
https://ksa.carswitch.com/en/saudi/used-car/hyundai/tucson/2018/132453-136115
https://ksa.carswitch.com/en/saudi/used-car/hyundai/tucson/2019/128605-132267
https://ksa.carswitch.com/en/saudi/used-car/hyundai/tucson/2017/12397-12996
https://ksa.carswitch.com/en/saudi/used-car/hyundai/tucson/2018/112331-112335
141 3340
'NoneType' object has no attribute 'findAll'


Time comsumed:  33.7973313331604


In [32]:
#df.to_csv('../Data/car_switch_data.csv',index=False)

In [9]:
df_all=pd.read_csv('../Data/car_switch_data.csv')

In [18]:
df_all.head()

,title,price,model_year,odo,aminities,specs,number_of_seats,drive_type,number_of_cylinders,body_type,...,first_owner,accidents_(not_verified),currently_financed?,seller_nationality,no._of_keys,roof_type,color,service_history,warranty,service_contract
0,Nissan X-Trail I4 2.5L,"\nSAR 54,000",2016,"72,000 KM","['Cruise Control', 'dynamic radar cruise contr...",GCC /Saudi Specs,5.0,2WD,4,SUV,...,Yes,No,No,India,2.0,NaN,NaN,NaN,NaN,NaN
1,Chrysler 300 V6 3.6L Limited,"\nSAR 30,000",2013,"207,000 KM",['front & rear park sensors front parking sens...,GCC /Saudi Specs,5.0,2WD,6,Sedan,...,NaN,3 or more accidents,No,Saudi Arabia,1.0,NaN,NaN,NaN,NaN,NaN
2,Nissan Altima I4 2.5L S,"\nSAR 20,500",2010,"273,000 KM",['front & rear park sensors front parking sens...,GCC /Saudi Specs,5.0,2WD,4,Sedan,...,NaN,No,No,Yemen,1.0,Sun Roof,NaN,NaN,NaN,NaN
3,GMC Acadia V6 3.6L,"\nSAR 22,000",2012,"135,000 KM",['front & rear park sensors front parking sens...,GCC /Saudi Specs,7.0,2WD,6,SUV,...,NaN,No,No,Egypt,2.0,NaN,NaN,NaN,NaN,NaN
4,Mazda 6 I4 2.5L,"\nSAR 55,000",2017,"87,000 KM",['front & rear park sensors front parking sens...,GCC /Saudi Specs,5.0,2WD,4,Sedan,...,Yes,No,No,Pakistan,1.0,NaN,Red,Yes in Garage,NaN,NaN


In [11]:
df_all.columns

Index(['title', 'price', 'model_year', 'odo', 'aminities', 'specs',
       'number_of_seats', 'drive_type', 'number_of_cylinders', 'body_type',
       'transmission_type', 'fuel_type', 'rim_size', 'first_owner',
       'accidents_(not_verified)', 'currently_financed?', 'seller_nationality',
       'no._of_keys', 'roof_type', 'color', 'service_history', 'warranty',
       'service_contract'],
      dtype='object')

In [13]:
df_all.shape

(3340, 23)

In [17]:
# check for duplication
df_all[df_all.duplicated()].shape

(47, 23)

In [25]:
df_all['aminities'][0]

"['Cruise Control', 'dynamic radar cruise control', 'Bluetooth', 'premium sound premium audio premium audio system', 'Power Lumbar', 'fog lamps fogdriving lights projector fog lamps', 'Emergency braking assist']"